In [86]:
import re
import pandas as pd
from pymongo import MongoClient
import folium
import googlemaps
from datetime import datetime
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [4]:
def filterType(lat,lng, typeName, radius):
    db, coll = connectCollection('companies', typeName)
    results = coll.find({
        "location":{
            "$near":{
                "$geometry":{
                    "type":"Point",
                    "coordinates":[lng,lat]
                },
                "$maxDistance": radius,
            }
        }
    })
    return results

coords = [52.3573865,4.892337]
data = filterType(*coords,'clubs',1000)
print(data.count())
print(data[0])


12
{'_id': ObjectId('5dd6ada72eb127ee9342a41c'), 'name': "Oostheim Bro's", 'address': 'Albert Cuypstraat 139, 1072 CS Amsterdam, Netherlands', 'location': {'type': 'Point', 'coordinates': [4.8927191, 52.3555603]}}


/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.


In [19]:
db, localrent = connectCollection('companies', 'rentlocals')
db, clubs = connectCollection('companies', 'clubs')

In [31]:
lr = list(localrent.find())
lr[0]

{'_id': ObjectId('5dd6ca627e4ead4e4526151a'),
 'address': 'Corsicaweg 10',
 'location': {'type': 'Point', 'coordinates': [4.8115896, 52.4021632]}}

In [44]:
def counting(collection, radius):
    lst=[]
    for l in lr:
        lst.append(l['location']['coordinates'][::-1])
    data=[]
    for l in lst:
        data.append(filterType(*l,collection,radius))
    contando=[]
    for i in range(len(data)):
        contando.append(data[i].count())
    return contando

In [50]:
clubs = counting('clubs',1000)
print(clubs)

[0, 0, 6, 3, 0, 0, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 12, 10, 1, 20, 0, 3, 2, 6, 3, 0, 0]


/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  # Remove the CWD from sys.path while we load stuff.


In [55]:
schools = counting('schools', 750)
print(schools)

[0, 1, 0, 3, 0, 0, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 4, 2, 1, 1, 0, 3, 0, 5, 0, 1, 0]


/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  # Remove the CWD from sys.path while we load stuff.


In [65]:
starbucks = counting('starbucks',200)
print(starbucks)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  # Remove the CWD from sys.path while we load stuff.


In [66]:
veganrest = counting('veganrest', 150)
print(veganrest)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  # Remove the CWD from sys.path while we load stuff.


In [62]:
kindergarten = counting('kindergarten', 600)
print(kindergarten)

[0, 1, 2, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 3, 0, 1, 0, 0, 0]


/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  # Remove the CWD from sys.path while we load stuff.


In [69]:
amsterdam = counting('amsterdam', 750)
print(amsterdam)

[0, 0, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 7, 0, 13, 0, 6, 1, 1, 0, 4, 0]


/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  # Remove the CWD from sys.path while we load stuff.


In [112]:
airport = counting('airport', 15000)
print(airport)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  # Remove the CWD from sys.path while we load stuff.


In [113]:
df=pd.DataFrame(lr)
df.head()

,_id,address,location
0,5dd6ca627e4ead4e4526151a,Corsicaweg 10,"{'type': 'Point', 'coordinates': [4.8115896, 5..."
1,5dd6ca627e4ead4e4526151b,Robert Kochplantsoen 19,"{'type': 'Point', 'coordinates': [4.9369403, 5..."
2,5dd6ca627e4ead4e4526151c,Eerste Leliedwarsstraat 6,"{'type': 'Point', 'coordinates': [4.8824977999..."
3,5dd6ca627e4ead4e4526151d,Geuzenstraat 98,"{'type': 'Point', 'coordinates': [4.8648476, 5..."
4,5dd6ca627e4ead4e4526151e,Jan Rebelstraat 18C,"{'type': 'Point', 'coordinates': [4.7859449, 5..."


In [114]:
def insertColumn(name, listname):
    df.insert(2, name, listname) 
    return df

insertColumn('airport', airport)
insertColumn('amsterdam', amsterdam)
insertColumn('clubs', clubs)
insertColumn('veganrest', veganrest)
insertColumn('starbucks', starbucks)
insertColumn('schools', schools)
insertColumn('kindergarten', kindergarten)
insertColumn('coordinates', lst)




,_id,address,coordinates,kindergarten,schools,starbucks,veganrest,clubs,amsterdam,airport,location
0,5dd6ca627e4ead4e4526151a,Corsicaweg 10,"[52.4021632, 4.8115896]",0,0,0,0,0,0,1,"{'type': 'Point', 'coordinates': [4.8115896, 5..."
1,5dd6ca627e4ead4e4526151b,Robert Kochplantsoen 19,"[52.3480536, 4.9369403]",1,1,0,0,0,0,1,"{'type': 'Point', 'coordinates': [4.9369403, 5..."
2,5dd6ca627e4ead4e4526151c,Eerste Leliedwarsstraat 6,"[52.3759084, 4.882497799999999]",2,0,0,0,6,14,1,"{'type': 'Point', 'coordinates': [4.8824977999..."
3,5dd6ca627e4ead4e4526151d,Geuzenstraat 98,"[52.37328549999999, 4.8648476]",1,3,0,0,3,0,1,"{'type': 'Point', 'coordinates': [4.8648476, 5..."
4,5dd6ca627e4ead4e4526151e,Jan Rebelstraat 18C,"[52.3614967, 4.7859449]",0,0,0,0,0,0,1,"{'type': 'Point', 'coordinates': [4.7859449, 5..."
5,5dd6ca627e4ead4e4526151f,Willebroekstraat 62,"[52.344418, 4.805356700000001]",0,0,0,0,0,0,1,"{'type': 'Point', 'coordinates': [4.8053567000..."
6,5dd6ca627e4ead4e45261520,Admiraal de Ruijterweg 402,"[52.3827785, 4.8512083]",1,1,0,0,1,0,1,"{'type': 'Point', 'coordinates': [4.8512083, 5..."
7,5dd6ca627e4ead4e45261521,Geuzenstraat 98,"[52.37328549999999, 4.8648476]",1,3,0,0,3,0,1,"{'type': 'Point', 'coordinates': [4.8648476, 5..."
8,5dd6ca627e4ead4e45261522,Nieuw-Zeelandweg 6E,"[52.40846759999999, 4.8022819]",0,0,0,0,0,0,1,"{'type': 'Point', 'coordinates': [4.8022819, 5..."
9,5dd6ca627e4ead4e45261523,Nieuw Zeelandweg 6,"[52.4081418, 4.8013445]",0,0,0,0,0,0,1,"{'type': 'Point', 'coordinates': [4.8013445, 5..."
